In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import tensorflow as tf
import math
import tensorflow_addons as tfa
import random
import re
import csv


from sklearn import metrics
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mlxtend.evaluate import mcnemar_table
from statsmodels.stats.contingency_tables import mcnemar




from utils import (
    F1Score,
    plot_metrics,
    plot_accuracy,
    study_oriented_transformation,
    write_csv,
    prediction_results,
    plot_confusion_matrix,
    plot_contigency_table,
)


In [2]:
# To Activate GPU if there is
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2022-09-25 23:56:25.678237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-25 23:56:25.719978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-25 23:56:25.720796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
SEED = 1037

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [4]:
METRICS = [ 
    tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
    F1Score(name="f1_score"),
]

STUDY_TYPES = [
    'XR_ELBOW',
    'XR_FINGER',
    'XR_FOREARM',
    'XR_HAND',
    'XR_HUMERUS',
    'XR_SHOULDER',
    'XR_WRIST',
]

CLASSES = ['NORMAL', 'ABNORMAL']

2022-09-25 23:56:25.837425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 23:56:25.840080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-25 23:56:25.840557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-25 23:56:25.840993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [5]:
data_directory = 'dataset'
test_img = pd.read_csv(os.path.join(data_directory, 'MURA-v1.1/test_image_paths.csv'), names=['path'])

In [6]:
test_img['label'] = test_img['path'].map(
    lambda x: '1' if 'positive' in x else '0'
)

test_img['study_type'] = test_img['path'].map(
    lambda x: x.split('/')[2]
)

test_img

,path,label,study_type
0,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
1,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
2,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
3,MURA-v1.1/test/XR_WRIST/patient11185/study1_po...,1,XR_WRIST
4,MURA-v1.1/test/XR_WRIST/patient11186/study1_po...,1,XR_WRIST
...,...,...,...
3192,MURA-v1.1/test/XR_FINGER/patient11967/study1_n...,0,XR_FINGER
3193,MURA-v1.1/test/XR_FINGER/patient11967/study1_n...,0,XR_FINGER
3194,MURA-v1.1/test/XR_FINGER/patient11738/study1_n...,0,XR_FINGER
3195,MURA-v1.1/test/XR_FINGER/patient11738/study1_n...,0,XR_FINGER


In [7]:
img_height = img_width = 300

def resize_img(img):
    try:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    except:
        print('error in resizing')
        img1 = isinstance(img, type(None))
        print('Does image is none: ', img1)
        print(img.shape)
    return cv2.resize(img, (img_height, img_width))

def canny_cropping(img):
    convert_img = np.array(img, dtype=np.uint8)

    gray = cv2.cvtColor(convert_img, cv2.COLOR_RGB2GRAY)


    ave_brightness = math.floor(np.average(gray))
    min_pixel = min(gray.flatten())

    edges = cv2.Canny(gray, min_pixel, ave_brightness)
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        x, y, w, h = cv2.boundingRect(edges)
        gray = gray[y:y+h, x:x+w]
        break

    return gray

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img.astype(np.uint8))

def preprocessing_without_clahe(img):
    cropped = canny_cropping(img)
    return resize_img(cropped)

def preprocessing_with_clahe(img):
    cropped = canny_cropping(img)
    clahe = apply_clahe(cropped)
    return resize_img(clahe)


# Evaluation of Model Without CLAHE

In [8]:
batch = 8
data_path = 'dataset/'

test_batches_without_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_without_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=test_img, 
    directory= data_path,
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 3197 validated image filenames belonging to 2 classes.


In [9]:
model_without_clahe = tf.keras.models.load_model(
    'models/without_clahe/efficientnetv2-b3_finetuned.h5',
    custom_objects={'F1Score': F1Score}
)

In [10]:
eval_without_clahe = model_without_clahe.evaluate(
    test_batches_without_clahe, 
    verbose = 1
)


2022-09-25 23:56:37.130231: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-25 23:56:38.057231: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


400/400 [==============================] - 38s 81ms/step - loss: 0.4926 - binary_accuracy: 0.8148 - precision: 0.8399 - recall: 0.7575 - cohen_kappa: 0.6275 - f1_score: 0.7966


In [11]:
eval_results = []

loss = eval_without_clahe.pop(0)

eval_results.append({'metric': 'test loss', 'value': loss})

# print('=' * 32)
# print(f'test loss   : {loss}')

for metric, score in zip(METRICS, eval_without_clahe):
    eval_results.append({'metric': metric.name, 'value': score})
    # print(f"{metric.name + ' ' * (12 - len(metric.name))}: {score}")

write_csv(
    eval_results, 
    'testing_results/without_clahe/evaluation_results.csv'
)

print('=' * 32)
for result in eval_results:
    print(f"{result['metric'] + ' ' * (12 - len(result['metric']))}: {result['value']}")
print('=' * 32)


test loss   : 0.4925764203071594
binary_accuracy: 0.8148264288902283
precision   : 0.8398550748825073
recall      : 0.7575163245201111
cohen_kappa : 0.6274706721305847
f1_score    : 0.7965636253356934


In [12]:
predictions_without_clahe = model_without_clahe.predict(test_batches_without_clahe, verbose=1)

400/400 [==============================] - 33s 78ms/step


In [13]:
conv_prediction_without_clahe = test_img.copy()

conv_prediction_without_clahe['label'] = conv_prediction_without_clahe['label'].map(int)
conv_prediction_without_clahe['prediction'] = predictions_without_clahe.ravel()

In [14]:
conv_prediction_without_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_without_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)

## Evaluation for all body parts concerned

In [15]:
results_without_clahe = prediction_results(conv_prediction_without_clahe)

write_csv(
    results_without_clahe, 
    'testing_results/without_clahe/general_prediction_results.csv'
)

print('=' * 52)
print('Prediction for all Musculoskeletal radiographs')
print('-----------------')
for result in results_without_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all Musculoskeletal radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6274706867671691
F1 Score                      : 0.7965635738831616
Accuracy                      : 0.8148263997497655
Precision                     : 0.8398550724637681
Recall                        : 0.757516339869281


In [16]:
cm_general_without_clahe = results_without_clahe[-1]['value']
print(cm_general_without_clahe)
plot_confusion_matrix(
    cm_general_without_clahe, 
    CLASSES, 
    'Confusion Matrix for all Musculoskeletal Parts',
    False,
)

[[1446  221]
 [ 371 1159]]


<Figure size 432x288 with 0 Axes>

## Evaluation per body types

In [17]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_without_clahe[conv_prediction_without_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results_without_clahe, 
        f'testing_results/without_clahe/{body_part}_prediction_results.csv'
    )
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'Confusion Matrix for {body_part}', 
        False
    )
    print('=' * 52)
    print(f'Prediction for {body_part} radiographs')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)


Prediction for XR_ELBOW radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6941781297762957
F1 Score                      : 0.8352668213457077
Accuracy                      : 0.8473118279569892
Precision                     : 0.8955223880597015
Recall                        : 0.782608695652174
Prediction for XR_FINGER radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.5627048602958441
F1 Score                      : 0.7855626326963906
Accuracy                      : 0.7809110629067245
Precision                     : 0.8258928571428571
Recall                        : 0.7489878542510121
Prediction for XR_FOREARM radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6347329170619773
F1 Score                      : 0.8028673835125448
Accuracy                      : 0.8172757475083057
Precision                     : 0.875
Recall                        : 0.7417218543046358
Prediction for XR_HAND radiographs
-----------------
Cohen's kappa Coefficient (κ) :

<Figure size 432x288 with 0 Axes>

# Evaluation of Model with CLAHE

In [18]:
batch = 8
data_path = 'dataset/'

test_batches_with_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_with_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=test_img, 
    directory= data_path,
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 3197 validated image filenames belonging to 2 classes.


In [19]:
model_with_clahe = tf.keras.models.load_model(
    'models/with_clahe/efficientnetv2-b3_finetuned.h5',
    custom_objects={'F1Score': F1Score}
)

In [20]:
eval_with_clahe = model_with_clahe.evaluate(
    test_batches_with_clahe, 
    verbose = 1
)

400/400 [==============================] - 36s 80ms/step - loss: 0.8139 - binary_accuracy: 0.7992 - precision: 0.8208 - recall: 0.7425 - cohen_kappa: 0.5961 - f1_score: 0.7797


In [21]:
with_clahe_eval_results = []

loss = eval_with_clahe.pop(0)

with_clahe_eval_results.append({'metric': 'test loss', 'value': loss})


for metric, score in zip(METRICS, eval_with_clahe):
    with_clahe_eval_results.append({'metric': metric.name, 'value': score})

write_csv(
    with_clahe_eval_results, 
    'testing_results/with_clahe/evaluation_results.csv'
)

print('=' * 32)
for result in with_clahe_eval_results:
    print(f"{result['metric'] + ' ' * (12 - len(result['metric']))}: {result['value']}")
print('=' * 32)


test loss   : 0.8139409422874451
binary_accuracy: 0.7991867661476135
precision   : 0.8208092451095581
recall      : 0.7424836754798889
cohen_kappa : 0.5960506200790405
f1_score    : 0.7796842455863953


In [22]:
predictions_with_clahe = model_with_clahe.predict(test_batches_with_clahe, verbose=1)

400/400 [==============================] - 34s 80ms/step


In [23]:
conv_prediction_with_clahe = test_img.copy()

conv_prediction_with_clahe['label'] = conv_prediction_with_clahe['label'].map(int)
conv_prediction_with_clahe['prediction'] = predictions_with_clahe.ravel()

In [24]:
conv_prediction_with_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_with_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)

In [25]:
results_with_clahe = prediction_results(conv_prediction_with_clahe)

write_csv(
    results_with_clahe, 
    'testing_results/with_clahe/general_prediction_results.csv'
)

print('=' * 52)
print('Prediction for all Musculoskeletal radiographs')
print('-----------------')
for result in results_with_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all Musculoskeletal radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.596050633947764
F1 Score                      : 0.7796842827728209
Accuracy                      : 0.7991867375664685
Precision                     : 0.8208092485549133
Recall                        : 0.742483660130719


In [26]:
cm_general_with_clahe = results_with_clahe[-1]['value']
print(cm_general_with_clahe)
plot_confusion_matrix(
    cm_general_with_clahe, 
    CLASSES, 
    'Confusion Matrix for all Musculoskeletal Parts',
    True,
    )

[[1419  248]
 [ 394 1136]]


<Figure size 432x288 with 0 Axes>

In [27]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_with_clahe[conv_prediction_with_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results_with_clahe, 
        f'testing_results/with_clahe/{body_part}_prediction_results.csv'
    )
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'Confusion Matrix for {body_part}', 
        True
    )
    print('=' * 52)
    print(f'Prediction for {body_part} radiographs')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)

Prediction for XR_ELBOW radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6943197074209527
F1 Score                      : 0.8390022675736961
Accuracy                      : 0.8473118279569892
Precision                     : 0.8767772511848341
Recall                        : 0.8043478260869565
Prediction for XR_FINGER radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.4642850448896969
F1 Score                      : 0.7327586206896551
Accuracy                      : 0.7310195227765727
Precision                     : 0.783410138248848
Recall                        : 0.6882591093117408
Prediction for XR_FOREARM radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0.6347973792768746
F1 Score                      : 0.7970479704797048
Accuracy                      : 0.8172757475083057
Precision                     : 0.9
Recall                        : 0.7152317880794702
Prediction for XR_HAND radiographs
-----------------
Cohen's kappa Coefficient (κ) : 0

<Figure size 432x288 with 0 Axes>

# Compare the two models using McNemar's Test

In [28]:
comparison = mcnemar_table(
    y_target= np.array(conv_prediction_without_clahe['label']),
    y_model1= np.array(conv_prediction_without_clahe['prediction']),
    y_model2= np.array(conv_prediction_with_clahe['prediction']),
)
print(comparison)
plot_contigency_table(
    comparison, 
    ['Correct', 'Wrong'],
    'Contigency Table for Both Models'
)

[[2378  227]
 [ 177  415]]


<Figure size 432x288 with 0 Axes>

In [31]:
mncnemar_without_correction = mcnemar(comparison, exact=False)
mncnemar_with_correction = mcnemar(comparison, exact=False, correction=True)

print('=' * 35)
print('McNemar\'s test without correction')
print(mncnemar_without_correction)
print('=' * 35)
print('McNemar\'s test with correction')
print(mncnemar_with_correction)

McNemar's test without correction
pvalue      0.01477526655224646
statistic   5.943069306930693
McNemar's test with correction
pvalue      0.01477526655224646
statistic   5.943069306930693
